In [1]:
library(tidyverse)
library(stats)
library(data.table)
library(pROC)
library(betareg)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth,

In [ ]:
# load data
data_dir <- "/projects/zhanglab/users/ana/multiome/validation/gentruth/model/train/" 
gwas_pops <- data_dir %>%
    paste0("pops_eval.tsv.gz") %>% 
    read_tsv() %>% 
    data.frame() %>% 
    dplyr::select(region, chr, start, gene, pli, cts, dist, pops_score, trait) %>% 
    dplyr::mutate(gwas=1) %>%
    dplyr::group_by(trait, region) %>%
    dplyr::mutate(
        pops_priority = if_else(pops_score == max(pops_score), 1, 0) # all snps since score is gene level
    ) %>%
    dplyr::ungroup()

multiome_pgb <- data_dir %>% 
    paste0("pgb_constituents.tsv.gz") %>% 
    read_tsv() %>% 
    data.frame() %>% 
    dplyr::select(region, chr, start, gene, pli, cts, dist, SCENT) %>% 
    dplyr::mutate(
        multiome=1, 
        multiome_finemap = if_else(SCENT<=0.05, 1, 0)
    )

eqtl_gtex_susie <- data_dir %>% 
    paste0("gtex_susie.tsv.gz") %>% 
    read_tsv() %>% 
    data.frame() %>% 
    dplyr::select(region, chr, start, gene, pli, cts, dist, pip, tissue) %>% 
    dplyr::mutate(eqtl=1) %>% 
    group_by(tissue, gene) %>%
    arrange(desc(pip)) %>%
    mutate(
        cumulative_pip = cumsum(pip),
        credible_set = if_else(
            (cumulative_pip <= 0.95 | row_number() == 1), 1, 0
            # (cumulative_pip <= 0.95 | row_number() == 1) & row_number() <= 10, 1, 0
        ), 
        max_pip = if_else(row_number() == 1, 1, 0)
    ) %>%
    ungroup()
# merge to add indicator variables
intersection_set <- eqtl_gtex_susie %>% 
    dplyr::inner_join(
        multiome_pgb %>% 
            dplyr::select(chr, start, gene, multiome, multiome_finemap) %>% 
            distinct(chr, start, gene, .keep_all = TRUE),
        by=c("chr", "start", "gene")
    ) %>% 
    dplyr::inner_join(
        gwas_pops %>% 
            dplyr::select(chr, start, gene, gwas, pops_priority, trait) %>% 
            distinct(chr, start, gene, .keep_all = TRUE),
        by=c("chr", "start", "gene")
    ) %>% 
    dplyr::mutate_all(~replace(., is.na(.), 0))

print(dim(intersection_set))
print(dim(eqtl_gtex_susie))
print(dim(multiome_pgb))
print(dim(gwas_pops))

New names:
• `` -> `...1`
Rows: 246884 Columns: 151
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (21): region, chr, gene, trait, gene_region, variant, gene_symbol, Featu...
dbl (54): ...1, start, end, cs_id, chromosome, gene_start, gene_end, tss, co...
lgl (76): gold, coloc_prior, coding_prior, abc_prior, popsexpr_prior, twas_p...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 3893164 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (5): rsID, Gene_symbol, gene, chr, region
dbl (15): ...1, SCENT, SCENT_percentile, Signac, Signac_percentile, ArchR, A...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`